# Grid Fluid

Modeling fluid motion by solving the incompressible and inviscid Navier-Stokes equation on a uniform grid

Author: Andres Ibarra, written as part of 22F Leave Term Research

Credit to Bo Zhu, on whose Dartmouth COSC89 course notes the following code has been based.

Additionally, credit to Robert Bridson from the University of British Columbia, whose Fluid Simulation Course Notes were invaluable in understanding the material. His course notes can be found here:\
https://www.cs.ubc.ca/~rbridson/fluidsimulation/fluids_notes.pdf

In [30]:
import taichi as ti
import taichi.math as tm
import numpy as np

ti.init()

[Taichi] Starting on arch=x64


In [31]:
### Global Variables ###

iters = 40 # number of gauss-seidel iterations
dt = 0.03
res = 300 # screen size
dx = 1.0

vel = ti.Vector.field(2, float, shape=(res, res))
vel_copy = ti.Vector.field(2, float, shape=(res,res))
vel_divs = ti.field(float, shape=(res, res))
pressure_hat = ti.field(float, shape=(res, res))
smoke_density = ti.field(float, shape=(res,res))
den_copy = ti.field(float, shape=(res,res))
vor = ti.field(float, shape=(res,res))
N = ti.Vector.field(2, float, shape=(res, res))

init_vel = tm.vec2(0.5,0)
src_rad = 0.1
velocity_enhancer = 50

colors = ti.Vector.field(3, float, shape=(res,res)) 
debug = False

# tester variables
lap_p = ti.field(float, shape=(res, res))
vel_check = ti.Vector.field(2, float, shape=())
vor_check = ti.field(float, shape=())
f_N_check = ti.Vector.field(2, float, shape=())
pressure_check = ti.field(float, shape=())
div_check = ti.field(float, shape=())
lap_check = ti.field(float, shape=())

In [32]:
### Helper Functions ###

@ti.func 
def gradient_pressure(i:ti.i32, j:ti.i32):
    """Returns the gradient of the pressure at grid node (i, j)."""
    pressures = tm.vec2(pressure_hat[i+1, j] - pressure_hat[i-1, j], pressure_hat[i, j+1] - pressure_hat[i, j-1])
    return pressures / (2 * dx)

@ti.func
def divergence_vel(i:ti.i32, j:ti.i32):
    """Returns the divergence of the velocity at grid node (i, j)."""
    return (vel[i+1, j][0] - vel[i-1, j][0] + vel[i, j+1][1] - vel[i, j-1][1]) / (2 * dx)

@ti.func
def curl_vel(i:ti.i32, j:ti.i32):
    """Returns the curl of the velocity at grid node (i, j)."""
    return (vel[i+1, j][1] - vel[i-1, j][1] - vel[i, j+1][0] + vel[i, j-1][0]) / (2 * dx)

@ti.func
def laplacian_pressure(i:ti.i32, j:ti.i32):
    """Returns the laplacian of the pressure at grid node(i,j)."""
    return (pressure_hat[i+1, j] + pressure_hat[i-1, j] + pressure_hat[i, j+1] + pressure_hat[i, j-1] - 4 * pressure_hat[i, j]) / (dx * dx)

@ti.func
def isBoundary(row:ti.i32, col:ti.i32) -> bool:
    """Returns if node at (row, col) is on a screen boundary."""
    return (row == 0 or col == 0 or row == res-1 or col == res-1)

In [33]:
### Simulator Functions ###

@ti.func
def clamp_pos(pos):
    """Ensures that vector pos is within the screen."""
    epsilon = dx * 1e-3
    clamped_pos = tm.vec2(pos[0], pos[1])
    if clamped_pos[0] < 0:
        clamped_pos[0] = epsilon
    if clamped_pos[1] < 0:
        clamped_pos[1] = epsilon
    if clamped_pos[0] >= res:
        clamped_pos[0] = res-epsilon
    if clamped_pos[1] >= res:
        clamped_pos[1] = res-epsilon
    return clamped_pos

@ti.func
def needs_clamping(x1, x2, y1, y2):
    return x1 < 0 or x2 < 0 or y1 < 0 or y2 < 0 or x1 >= res or x2 >= res or y1 >= res or y2 >= res

@ti.func
def interpolate(x_p, vector_field):
    """Returns the bilinearly interpolated of the given field at position x_p."""
    x_p = clamp_pos(x_p)
    x = x_p[0]
    y = x_p[1]
    x1 = (int)(ti.floor(x))
    x2 = x1+1
    y1 = (int)(ti.floor(y))
    y2 = y1+1
    # clamp pos
    if needs_clamping(x1, x2, y1, y2):
        if x1 < 0:
            x1 = 0
            x2 = 1
        if y1 < 0:
            y1 = 0
            y2 = 1
        if x2 >= res:
            x1 = res-2
            x2 = res-1
        if y2 >= res:
            y1 = res-2
            y2 = res-1
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * vector_field[x1, y1] + (x - x1)/(x2 - x1) * vector_field[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * vector_field[x1, y2] + (x - x1)/(x2 - x1) * vector_field[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    
    return f_xy

@ti.func
def fill_copies():
    """Fills vel_copy and den_copy."""
    for i in ti.grouped(vel):
        vel_copy[i] = vel[i]
        den_copy[i] = smoke_density[i]

@ti.kernel
def advect():
    """Advection step of simulation using a semi-Lagrangian method."""
    fill_copies()
    for row in range(res):
        for col in range(res):
            pos = tm.vec2(row, col)
            new_pos = pos - vel[row, col] * 0.5 * dt
            new_vel = interpolate(new_pos, vel_copy)
            new_pos = pos - new_vel * dt
            new_vel = interpolate(new_pos, vel_copy)
            vel[row, col] = new_vel
            
            new_den = interpolate(new_pos, den_copy)
            smoke_density[row, col] = new_den

@ti.func
def calc_divs():
    """Fills velocity divergence field."""
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vel_divs[row, col] = divergence_vel(row, col)
    
@ti.func 
def fill_lap_p():
    """Fills laplacian of pressure field for use in debugging."""
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                lap_p[row, col] = laplacian_pressure(row, col)
    
@ti.kernel
def project():
    """Projection step of simulation."""
    # step 1: calculate divergence on each node
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vel_divs[row, col] = divergence_vel(row, col)
                
    # step 2: solve the Poisson's equation -lap p= div u using the Gauss-Seidel iterations
    ti.loop_config(serialize=True)
    for i in range(iters):
        for row in range(res):
            for col in range(res):
                if not isBoundary(row, col):
                    coef_off_dia = 1.0 / (dx * dx)
                    coef_dia = 4.0 / (dx * dx)
                    off_diagonal = pressure_hat[row-1, col] + pressure_hat[row+1, col] + pressure_hat[row, col+1] + pressure_hat[row, col-1]
                    pressure_hat[row, col] = (-vel_divs[row, col] + off_diagonal * coef_off_dia) / coef_dia
    
    fill_lap_p() # for the debug output
    
    # step 3: correct velocity with the pressure gradient
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                grad_p = gradient_pressure(row, col)
                vel[row, col] -= grad_p
    
    calc_divs() # for the debug output

@ti.func
def cross(v, w):
    """Calculates the cross product of v and w.
    
    Args:
        v: a 2D vector in the xy plane
        w: a scalar (the z component of a 3D vector with only a z component)
        
    Returns:
        The 2D vector cross product
    """
    return tm.vec2(v[1] * w, -v[0] * w)

@ti.kernel
def vorticity_confinement():
    """Vorticity confinement step of simulation."""
    vor.fill(0.)
    # step 1: update vorticity
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vor[row, col] = curl_vel(row, col)

    # step 2: update N = (grad(|vor|)) / |grad(|vor|)|
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                N[row, col][0] = (abs(vor[row+1, col]) - abs(vor[row-1, col]))/ (2 * dx)
                N[row, col][1] = (abs(vor[row, col+1]) - abs(vor[row, col-1]))/ (2 * dx)
                N[row, col] /= (N[row, col].norm() + 1e-20)

    # step 3: calculate confinement force and use it to update velocity
    vor_conf_coef = 4.0 # 4.0
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                f = vor_conf_coef * dx * cross(N[row, col], vor[row, col])
                vel[row, col] += f * dt
    
@ti.kernel
def valid_pos(x:ti.f32, y:ti.f32) -> ti.i32:
    valToReturn = 0 # start at false
    if x < 1 and x >= 0 and y < 1 and y >= 0:
        valToReturn = 1
    return valToReturn
    
@ti.func
def mouse_valid(x, y):
    valToReturn = 1 # start at true
    if (x <= 0 or x >= res) and (y <= 0 or y >= res):
        valToReturn = 0
    return valToReturn
        
    
@ti.kernel
def source(mouse_data: ti.types.ndarray()):
    """Adds a new source of smoke using mouse data."""
    center = tm.vec2(mouse_data[2]/res, mouse_data[3]/res)
    direction = tm.vec2(mouse_data[0], mouse_data[1])
    direction *= velocity_enhancer # increase velocity to make it more noticeable
    for row in range(res):
        for col in range(res):
            if mouse_valid(mouse_data[2], mouse_data[3]) and (tm.vec2(row/res, col/res) - center).norm() < src_rad:
                vel[row, col] = direction
                smoke_density[row, col] = 1

@ti.kernel
def update_colors():
    """Updates the colors field according to smoke density."""
    for i in ti.grouped(colors):
        curr_den = smoke_density[i]
        colors[i] = tm.vec3(curr_den, curr_den, curr_den)

In [34]:
def substep(mouse_data):
    source(mouse_data)
    if debug and mouse_data[2] != 0 and mouse_data[3] != 0:
        print(mouse_data)
    advect()
    vorticity_confinement()
    project()
    update_colors()

In [35]:
### Setup ###
@ti.kernel
def init_grid():
    """Initializes fields to 0."""
    for i in ti.grouped(vel):
        vel[i] = init_vel
        N[i] = tm.vec2(0,0)
    smoke_density.fill(0.)
    vel_divs.fill(0.)
    pressure_hat.fill(0.)
    vor.fill(0.)

In [36]:
### Function to get print checks ###
@ti.kernel
def get_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
    smoke_check[None] = smoke_density[row, col]
    vel_check[None] = vel[row, col]
    vor_check[None] = vor[row, col]
    pressure_check[None] = pressure_hat[row, col]
    f_N_check[None] = N[row, col]
    div_check[None] = vel_divs[row, col]
    lap_check[None] = lap_p[row, col]

def print_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
    print("(row, col) = (%d, %d)" % (row, col))
    print("smoke check", smoke_check[None])
    print("vel check", vel_check[None])
    print("vor check", vor_check[None])
    print("N check", f_N_check[None])
    print("pressure check", pressure_check[None])
    print("div check", div_check[None])
    print("lap check", lap_check[None])

In [37]:
class MouseData:
    """Stores data on the mouse direction and position.
    
    Credit to @feisuzhu, on whose stable_fluid.py code this is based
    """
    def __init__(self):
        self.prev_mouse = None
    
    def __call__(self, gui):
        """Returns the mouse data.
        
        Returns:
            A numpy array, where information is stored as follows:
            [0:2]: normalized delta direction
            [2:4]: current mouse xy
        """
        mouse_data = np.zeros(5, dtype=np.float32)
        if gui.is_pressed(ti.GUI.LMB):
            mouse_xy = np.array(gui.get_cursor_pos(), dtype=np.float32) * res
            if self.prev_mouse is None:
                self.prev_mouse = mouse_xy
            else:
                mouse_dir = mouse_xy - self.prev_mouse
                mouse_dir = mouse_dir / (np.linalg.norm(mouse_dir) + 1e-5)
                mouse_data[0], mouse_data[1] = mouse_dir[0], mouse_dir[1]
                mouse_data[2], mouse_data[3] = mouse_xy[0], mouse_xy[1]
                self.prev_mouse = mouse_xy
        else:
            self.prev_mouse = None
        return mouse_data

In [38]:
def main():
    gui = ti.GUI('Grid Fluid', (res, res))
    md_gen = MouseData()
    
    visualize_smoke = True  #visualize density (default)
    visualize_vel = False  #visualize velocity
    visualize_div = False # visualize divergence
    visualize_curl = False # visualize curl
        
    init_grid()

    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == 'c':
                if valid_pos(e.pos[0], e.pos[1]):
                    get_checks(e.pos[0], e.pos[1])
                    print_checks(e.pos[0], e.pos[1])
            elif e.key == 'v': # show velocities
                visualize_vel = True
                visualize_smoke = False
                visualize_div = False
                visualize_curl = False
            elif e.key == 'b':
                visualize_smoke = True
                visualize_vel = False
                visualize_div = False
                visualize_curl = False
            elif e.key == 'g':
                visualize_smoke = False
                visualize_vel = False
                visualize_div = True
                visualize_curl = False
            elif e.key == 'h':
                visualize_smoke = False
                visualize_vel = False
                visualize_div = False
                visualize_curl = True
            elif e.key == 'r':
                init_grid()
             
        # Move stuff
        mouse_data = md_gen(gui)
        substep(mouse_data)

        # Draw grid
        if visualize_smoke:
            gui.set_image(colors)
        elif visualize_vel:
            gui.set_image(vel.to_numpy() * 0.035 + 0.5)
        elif visualize_div:
            gui.set_image(vel_divs.to_numpy() * 0.1 + 0.3)
        elif visualize_curl:
            gui.set_image(vor.to_numpy() * 0.1 + 0.5)
        gui.show()
        
    
if __name__ == '__main__':
    main()